In [40]:
import pickle
import pandas as pd

import numpy as np
import os
from ete3 import NCBITaxa

import ortholog_mapper
from dotenv import load_dotenv
from datetime import datetime

import ast
from tqdm import tqdm

from IPython.display import clear_output



In [41]:


PARENT_DATA_PATH = '../data'
DATA_PATH = '../data/genelab_data_in_csv_format.pickle'
FILENAME = '20231220.csv'



In [42]:
load_dotenv('../.env')

NEO4J_DATA = os.getenv('NEO4J_DATA')
node_dir = os.path.join(NEO4J_DATA, 'nodes')
rel_dir = os.path.join(NEO4J_DATA, 'relationships')


In [43]:
spoke_relevant_assays = ['RNA Sequencing (RNA-Seq)', 'DNA microarray', 'mass spectrometry', 'nucleotide sequencing', '16S']
organism_to_exclude = ['Viridiplantae', 'Fungi']

metadata_df = pd.read_csv(os.path.join(PARENT_DATA_PATH, FILENAME), header=1)

ncbi = NCBITaxa()
lineage_superkingdom = [None]*metadata_df.shape[0]
lineage_kingdom = [None]*metadata_df.shape[0]
for index, row in metadata_df.iterrows():
    try:
        tax_id = row['organism.term accession number'].split('/')[-1]
        lineage = ncbi.get_lineage(tax_id)
        lineage_names = ncbi.get_taxid_translator(lineage)
        for item in lineage_names:
            rank = list(ncbi.get_rank([item]).values())[0]
            if rank == 'superkingdom':
                lineage_superkingdom[index] = lineage_names[item]
            if rank == 'kingdom':
                lineage_kingdom[index] = lineage_names[item]
    except:
        continue
metadata_df.loc[:, 'lineage_superkingdom'] = lineage_superkingdom
metadata_df.loc[:, 'lineage_kingdom'] = lineage_kingdom

metadata_df = metadata_df[metadata_df['GL-processed']!='False']
metadata_df_ = metadata_df[metadata_df['study assay technology type'].isin(spoke_relevant_assays)]
metadata_df_without_plants_fungi = metadata_df_[~metadata_df_['lineage_kingdom'].isin(organism_to_exclude)]
metadata_df_without_plants_fungi = metadata_df_without_plants_fungi[metadata_df_without_plants_fungi.organism != 'Not Applicable']
metadata_df_without_plants_fungi.dropna(subset=['lineage_kingdom'], inplace=True)
metadata_df_without_plants_fungi.dropna(subset=['organism'], inplace=True)

metadata_df_without_plants_fungi_ = metadata_df_without_plants_fungi.dropna(subset=['material type.term accession number'])
metadata_df_without_plants_fungi_.loc[:,'material_type_id'] = metadata_df_without_plants_fungi_['material type.term accession number'].apply(lambda x:x.split('/')[-1].split('_')[0])

metadata_df_without_plants_fungi_after_uberon_filter = metadata_df_without_plants_fungi_[(metadata_df_without_plants_fungi_.material_type_id.str.startswith('D')) | 
                                   (metadata_df_without_plants_fungi_.material_type_id == 'UBERON') |
                                 (metadata_df_without_plants_fungi_['material_type_id'].str.contains('fma'))|
                                 (metadata_df_without_plants_fungi_['material_type_id'].str.contains('EHDA')) |
                                 (metadata_df_without_plants_fungi_['material_type_id'].str.contains('owl'))]


clear_output()

In [44]:
metadata_df_without_plants_fungi_after_uberon_filter_org_info = metadata_df_without_plants_fungi_after_uberon_filter[['accession', 'organism', 'organism.term accession number']]

metadata_df_without_plants_fungi_after_uberon_filter_org_info.loc[:,'org_id'] = metadata_df_without_plants_fungi_after_uberon_filter_org_info['organism.term accession number'].apply(lambda x:x.split('/')[-1])

metadata_df_without_plants_fungi_after_uberon_filter_org_info.drop(['organism.term accession number'], axis=1, inplace=True)
clear_output()
metadata_df_without_plants_fungi_after_uberon_filter_org_info


,accession,organism,org_id
0,OSD-1,Drosophila melanogaster,7227
1,OSD-100,Mus musculus,10090
3,OSD-102,Mus musculus,10090
11,OSD-111,Mus musculus,10090
12,OSD-111,Mus musculus,10090
...,...,...,...
254,OSD-83,Oryzias latipes,8090
255,OSD-83,Oryzias latipes,8090
256,OSD-83,Oryzias latipes,8090
258,OSD-87,Mus musculus,10090


In [45]:
%%time

with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

CPU times: user 939 ms, sys: 2.78 s, total: 3.72 s
Wall time: 8.79 s


In [46]:

mgene_list = []
for item in data:
    df = item['data']
    p_val_col = list(filter(None, map(lambda x:x if 'Adj.p.value_' in x else None, list(item['data'].columns))))
    df.dropna(subset=p_val_col, inplace=True)
    df = df[(df[p_val_col] < 0.05).any(axis=1)]
    if df.shape[0] > 0:
        try:
            mgene_df = df[['ENTREZID']]
            try:
                mgene_df.loc[:,'GENENAME'] = df[['GENENAME']]
            except:
                try:
                    mgene_df.loc[:,'GENENAME'] = df[['SYMBOL']]
                except:
                    mgene_df.loc[:,'GENENAME'] = ''
        except:
            continue
        mgene_df.loc[:,'accession'] = item['metadata']['accession']
        mgene_list.append(mgene_df)
    

clear_output()
mgene_df = pd.concat(mgene_list, ignore_index=True)
mgene_df = mgene_df.drop_duplicates(subset=['ENTREZID']).dropna(subset=['ENTREZID'])
print(mgene_df.shape)


(106451, 3)


In [47]:
mgene_with_org_df = pd.merge(mgene_df, metadata_df_without_plants_fungi_after_uberon_filter_org_info, on='accession').drop_duplicates(subset=['ENTREZID'])
mgene_with_org_df.loc[:, 'organism'] = mgene_with_org_df['organism'].str.replace('Homo Sapiens', 'Homo sapiens')

mgene_with_org_df.rename(columns={'ENTREZID':'identifier',
                                  'GENENAME':'name',
                                  'org_id':'taxonomy'
                                 }, inplace=True)
mgene_with_org_df.drop(['accession'], axis=1, inplace=True)


In [48]:
%%time
# ortholog_dbs = list(ortholog_mapper.get_ortholog_dbs())
mgene_with_org_df = ortholog_mapper.map_orthologs(mgene_with_org_df, "taxonomy", "identifier", "human_entrez_gene", ortholog_dbs=["JAX", "Ensembl"])



CPU times: user 5.45 s, sys: 1.38 s, total: 6.83 s
Wall time: 2min 7s


In [49]:

MGiG_df = mgene_with_org_df[['identifier', 'human_entrez_gene']]
MGiG_df.rename(columns={'identifier':'from', 'human_entrez_gene':'to'}, inplace=True)
MGiG_df = MGiG_df[MGiG_df.to!=''].drop_duplicates()

clear_output()

In [50]:
gene_df = mgene_with_org_df[['human_entrez_gene']]
gene_df.rename(columns={'human_entrez_gene':'identifier'}, inplace=True)
gene_df.drop_duplicates(inplace=True)
gene_df = gene_df[gene_df.identifier!='']

clear_output()

In [51]:
mgene_with_org_df.drop('human_entrez_gene', axis=1, inplace=True)
mgene_with_org_df.drop_duplicates(subset=['identifier'], inplace=True)


## Assay-MGene

In [52]:
today_date = datetime.today()
formatted_date = today_date.strftime("%Y-%d-%m")

study_assay_df = pd.read_csv(os.path.join(rel_dir, f'Study-PERFORMED_SpA-Assay_{formatted_date}.csv'))
assay_df = pd.read_csv(os.path.join(node_dir, f'Assay_{formatted_date}.csv'))
study_ids = list(map(lambda x:x['metadata']['accession'], data))

assay_mgene = []
for index, row in tqdm(study_assay_df.iterrows()):
    study_id = row['from']
    assay_id = row['to']
    factors_1 = ast.literal_eval(assay_df[assay_df.identifier==assay_id].factors_1.values[0])
    factors_2 = ast.literal_eval(assay_df[assay_df.identifier==assay_id].factors_2.values[0])
    factors_1_str = ' & '.join(factors_1)
    factors_2_str = ' & '.join(factors_2)
    data_index = np.where(study_id==np.array(study_ids))[0][0]
    df = data[data_index]['data']
    try:
        log_fc_col = f'Log2fc_({factors_1_str})v({factors_2_str})'
        adj_pval_col = f'Adj.p.value_({factors_1_str})v({factors_2_str})'
        log_fc = df[log_fc_col]
        adj_pval = df[adj_pval_col]
    except:
        try:
            log_fc_col = f'Log2fc_({factors_1_str})v({factors_2_str}) '
            adj_pval_col = f'Adj.p.value_({factors_1_str})v({factors_2_str}) '
            log_fc = df[log_fc_col]
            adj_pval = df[adj_pval_col]
        except:
            try:
                log_fc_col = f'Log2fc_({factors_1_str} )v( {factors_2_str})'
                adj_pval_col = f'Adj.p.value_({factors_1_str} )v( {factors_2_str})'
                log_fc = df[log_fc_col]
                adj_pval = df[adj_pval_col]
            except:
                try:
                    log_fc_col = f'Log2fc_({factors_1_str} )v( {factors_2_str} )'
                    adj_pval_col = f'Adj.p.value_({factors_1_str} )v( {factors_2_str} )'
                    log_fc = df[log_fc_col]
                    adj_pval = df[adj_pval_col]
                except:
                    try:
                        log_fc_col = f'Log2fc_({factors_1_str})v({factors_2_str} )'
                        adj_pval_col = f'Adj.p.value_({factors_1_str})v({factors_2_str} )'
                        log_fc = df[log_fc_col]
                        adj_pval = df[adj_pval_col]
                    except:
                        try:
                            log_fc_col = f'Log2fc_({factors_1_str} )v({factors_2_str})'
                            adj_pval_col = f'Adj.p.value_({factors_1_str} )v({factors_2_str})'
                            log_fc = df[log_fc_col]
                            adj_pval = df[adj_pval_col]
                        except:
                            log_fc_col = f'Log2fc_({factors_1_str} )v({factors_2_str} )'
                            adj_pval_col = f'Adj.p.value_({factors_1_str} )v({factors_2_str} )'
                            log_fc = df[log_fc_col]
                            adj_pval = df[adj_pval_col]
        
    
    
    assay_mgene_df = df[['ENTREZID']]
    assay_mgene_df.loc[:,'log_fc'] = log_fc
    assay_mgene_df.loc[:,'adj_pval'] = adj_pval
    assay_mgene_df.loc[:,'assay_id'] = assay_id
    assay_mgene.append(assay_mgene_df)

clear_output()


In [53]:
%%time

AmMG_df = pd.concat(assay_mgene, ignore_index=True)
AmMG_df = AmMG_df[AmMG_df.ENTREZID.isin(mgene_with_org_df.identifier)]

AmMG_df = AmMG_df[AmMG_df.adj_pval<0.05].drop_duplicates(subset=['ENTREZID', 'assay_id'])

AmMG_df.loc[AmMG_df.log_fc>0, 'direction'] = 'upregulates'
AmMG_df.loc[AmMG_df.log_fc<0, 'direction'] = 'downregulates'

AmMG_df.rename(columns={'assay_id':'from', 'ENTREZID':'to', 'log_fc':'log2fc', 'adj_pval':'adj_p_value'}, inplace=True)
AmMG_df = AmMG_df[['from', 'to', 'log2fc', 'adj_p_value', 'direction']]


CPU times: user 5.26 s, sys: 2.28 s, total: 7.54 s
Wall time: 10.3 s


In [61]:
mgene_with_org_df.identifier = mgene_with_org_df.identifier.astype('str')
gene_df.identifier = gene_df.identifier.astype('str')
MGiG_df['from'] = MGiG_df['from'].astype('str')
MGiG_df['to'] = MGiG_df['to'].astype('str')
AmMG_df['from'] = AmMG_df['from'].astype('str')
AmMG_df['to'] = AmMG_df['to'].astype('str')


mgene_with_org_df.drop_duplicates(subset=['identifier'], inplace=True)
gene_df.drop_duplicates(subset=['identifier'], inplace=True)

AmMG_df = AmMG_df[AmMG_df['to'].isin(mgene_with_org_df.identifier)]
AmMG_df.drop_duplicates(inplace=True)

MGiG_df.drop_duplicates(inplace=True)





## Saving node and edge files

In [ ]:


mgene_with_org_df.to_csv(os.path.join(node_dir, f'MGene_{formatted_date}.csv'), index=False)
gene_df.to_csv(os.path.join(node_dir, f'Gene_{formatted_date}.csv'), index=False)

MGiG_df.to_csv(os.path.join(rel_dir, f'MGene-IS_ORTHOLOG_MGiG-Gene_{formatted_date}.csv'), index=False)
AmMG_df.to_csv(os.path.join(rel_dir, f'Assay-MEASURED_AmMG-MGene_{formatted_date}.csv'), index=False)

